# Use pulp to solve a wedding seating problem

We have a list of wedding guests who have a strange desire to be seated alone, or otherwise alphabetically as closely as possible. There are 5 tables and 4 seats at each. This notebook shows how to use the PuLP linear problem library to come up with the best seating arrangement.

In [173]:
import pulp

In [174]:
max_tables = 5
max_table_size = 4
guests = 'Ang Bai Cat Don Ed Fin Gru Ida Jo Koa Li Min Nan Odo Pip Q Rip'.split()

### Unhappiness function
The unhappiness of the table is defined as the largest distance between the first and last "guests". For simplicity, the function looks only at the first letter of each guest name and assumes that the list is already sorted.

This was called 'happiness' in the original example, but since we're supposed to minimize this value, it should really be called 'unhappiness'.  I found this extremely confusing until I realized the function name was just bad.  #namingishard?

In [175]:
def unhappiness(table):
    return abs(ord(table[0][0]) - ord(table[-1][0]))

**generate possible seating arrangements**, and display one of the tables to illustrate one

In [176]:
combos = pulp.allcombinations(guests, max_table_size)
possible_tables = [tuple(c) for c in combos]

print(possible_tables[245])

('Ang', 'Koa', 'Li')


### create a binary variable dictionary to indicate if a table setting is used

[LpVariable.dicts](https://coin-or.github.io/pulp/technical/pulp.html?highlight=dicts#pulp.LpVariable.dicts), a class method, creates a dictionary of LpVariable with the specified associated parameters

Each possible table setting will eventually have a not-used/used value of 0 or 1.  This method simply creates a dict with:
* keys = the values of the possible table seatings, 
* values = LpVariables with a variable name of table_&lt;key&gt;, and which can be either 0 or 1.


In [177]:
x = pulp.LpVariable.dicts('table', possible_tables, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

nthKey=list(x)[245]
print("Example key: ")
print(nthKey)
print("\nThe nth key in the dict x: ")
print(x[nthKey])

Example key: 
('Ang', 'Koa', 'Li')

The nth key in the dict x: 
table_('Ang',_'Koa',_'Li')


### Create the seating model
create a new LpProblem called "WeddingSeatingModel", whose goal is to minimize the alphabetic distance between the guests.

In [178]:
seating_model = pulp.LpProblem("WeddingSeatingModel", pulp.LpMinimize)

### Set the objective

Adding a _value_ to model makes this the objective.
Only one objective may be added to a model.
Since we said this was an LpMinimize type, the objective is to minimize this value.

`pulp.lpSum(vector)` Calculates the sum of a list of linear expressions.

Parameters:  `vector` – A list of linear expressions

In [179]:
seating_model += pulp.lpSum([unhappiness(table) * x[table] for table in possible_tables])

### Set constraints
Adding an _expression_  adds a constraint. The solution may not violate constraints.
We specify the sensible constraint that the solution has to be <= some limit.

In [180]:
seating_model += pulp.lpSum([x[table] for table in possible_tables]) <= max_tables,"Maximum_number_of_tables"

Now we need to ensure that guests are not duplicated. They can sit at one table only. Add a constraint for each guest.

In [181]:
for guest in guests:
    seating_model += pulp.lpSum([x[table] for table in possible_tables
                                if guest in table]) == 1, "Must_seat_%s"%guest

### Tell the model to solve

In [182]:
solution_status = seating_model.solve()

### Show the solution

In [183]:
if solution_status < 0:
    print("the problem is infeasible")
else:
    print("The chosen tables are out of a total of %s:"%len(possible_tables))
    for table in possible_tables:
        if x[table].value() > 0:
            print(table)

The chosen tables are out of a total of 3213:
('Min', 'Nan')
('Ed', 'Fin', 'Gru')
('Ang', 'Bai', 'Cat', 'Don')
('Ida', 'Jo', 'Koa', 'Li')
('Odo', 'Pip', 'Q', 'Rip')


We can then play around with the table size and number of tables to see how this affects the solution, and easily rerun the cells to show the answer.